In [1]:
import os 
import pathlib

import numpy as np

import dask
from dask import diagnostics
import dask.array as da

from skimage.io import imread as skimread

import SessionTools as st 
import SessionTools.two_photon



%load_ext autoreload
%autoreload 2 

In [2]:
basedir = pathlib.PurePath('/media/mplitt/SSD_storage/2PData/12_16_2022/EPG_GCaMP7f_CsChrimsontdT_fly1/fixed_cw_remapping-000/')
session = "fixed_cw_remapping-000"
basename_input = pathlib.Path(basedir.joinpath(session))

metadata = st.two_photon.metadata.read(basename_input,pathlib.Path('.'))

In [3]:
metadata['size']
metadata['layout']

{'sequences': 3280, 'frames_per_sequence': 7, 'samples_per_pixel': 6}

In [ ]:
tiff_data = st.two_photon.tiff_tools.read(basename_input, 
                                          metadata['size'],metadata['layout'],
                                          2)

In [4]:
channel = 2
filenames = []
num_cycles = metadata['layout']['sequences']
for cycle in range(num_cycles):
    for frame in range(metadata['layout']['frames_per_sequence']):
        filenames.append(str(basename_input) + f'_Cycle{cycle+1:05d}_Ch{channel}_{frame+1:06d}.ome.tif')

In [5]:
def read_one_image(block_id, filenames=filenames[:10], axis=0):
    print(block_id)
    path = filenames[block_id[axis]]
    image = skimread(path)
    return np.expand_dims(image, axis=axis)


In [6]:
sample = skimread(filenames[0])
stack = da.map_blocks(
    read_one_image,
    dtype=sample.dtype,
    chunks=((1,)*len(filenames), *sample.shape)
)

# now figure out how to add cycles

<tifffile.read_uic_tag> failed reading CreateTime with ValueError: no datetime before year 1 (julianday=0)
<tifffile.read_uic_tag> failed reading LastSavedTime with ValueError: no datetime before year 1 (julianday=0)
<tifffile.read_uic_tag> failed reading CreateTime with ValueError: no datetime before year 1 (julianday=0)
<tifffile.read_uic_tag> failed reading LastSavedTime with ValueError: no datetime before year 1 (julianday=0)


[]


In [7]:
stack.compute()

<tifffile.read_uic_tag> failed reading CreateTime with ValueError: no datetime before year 1 (julianday=0)
<tifffile.read_uic_tag> failed reading LastSavedTime with ValueError: no datetime before year 1 (julianday=0)
<tifffile.read_uic_tag> failed reading CreateTime with ValueError: no datetime before year 1 (julianday=0)
<tifffile.read_uic_tag> failed reading LastSavedTime with ValueError: no datetime before year 1 (julianday=0)


(0, 0, 0)
(22638, 0, 0)
(2150, 0, 0)
(9567, 0, 0)
(1456, 0, 0)
(17745, 0, 0)
(6600, 0, 0)
(554, 0, 0)
(9895, 0, 0)
(10919, 0, 0)
(17736, 0, 0)
(19662, 0, 0)
(18760, 0, 0)
(11388, 0, 0)
(3069, 0, 0)
(10486, 0, 0)
(1473, 0, 0)
(7943, 0, 0)
(571, 0, 0)
(10814, 0, 0)
(22646, 0, 0)
(18655, 0, 0)
(20581, 0, 0)
(17753, 0, 0)


IndexError: list index out of range

In [ ]:
stack.shape

In [ ]:
(1,)*3

In [ ]:
base = basename_input
cycle = 0
channel = 2
frame = 1
fname = str(base) + f'_Cycle{cycle+1:05d}_Ch{channel}_{frame+1:06d}.ome.tif'

In [ ]:
os.path.exists(fname)

In [ ]:
dtype = skimread(fname).dtype

In [ ]:
imread = dask.delayed(skimread, pure=True)

In [ ]:
lazy_image = imread(fname)
sample = lazy_image.compute()
arr = da.from_delayed(lazy_image, dtype=sample.dtype, shape=sample.shape)

In [ ]:

lazy_image = dask.delayed(imread)(fname)
arr = da.from_delayed(lazy_image, (metadata['size']['y_px'], metadata['size']['x_px']), dtype=dtype)

In [ ]:
arr = lazy_image.compute()

In [ ]:
arr.shape